In [ ]:
#this script is compatible with google cloud functions API
import logging
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

OPENAI_API_KEY = ""
index_name = "neural"
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, openai_api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

@functions_framework.http
def generate(request):
    logger.info("Entered the generate function")

    if request.method == "OPTIONS":
        headers = {
            "Access-Control-Allow-Origin": "*",
            "Access-Control-Allow-Methods": "POST, OPTIONS",
            "Access-Control-Allow-Headers": "Content-Type",
            "Access-Control-Max-Age": "3600",
        }
        return "", 204, headers

    request_json = request.get_json(silent=True)
    if not request_json or 'query' not in request_json:
        logger.warning("No 'query' field in the request!")
        return "No 'query' field in your request!", 400, {"Access-Control-Allow-Origin": "*"}
    
    query = request_json['query']
    logger.info(f"Received query: {query}")

    # Perform a similarity search on the vector store
    logger.info("Performing similarity search")
    search_results = vectorstore.similarity_search(query, k=1)  # Top 3 similar documents
    clean_results = "\n\n".join([doc.page_content for doc in search_results])

    # Generate a response based on the retrieved documents
    prompt = [
        {"role": "system", "content": "You are an AI assistant and you have to provide the answer based only on the context provided: {clean_results}"},
        {"role": "user", "content": query}
    ]
    response = llm(
        messages=f"Based on the information in the following documents: {clean_results}, respond to the query: {query}"
    )

    response = response.content

    headers = {"Access-Control-Allow-Origin": "*"}  
    return {"response": response}, 200, headers